In [1]:
import os
import sys
project_root = os.path.abspath('../src')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import wandb
key = None
if key is not None:
    wandb.login(key=key)
    os.environ['WANDB_API_KEY'] = key
os.environ["WANDB_NOTEBOOK_NAME"] = "capture24.ipynb"

In [2]:
!nvidia-smi

Fri May 23 01:30:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.08             Driver Version: 550.127.08     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-32GB           Off |   00000000:06:00.0 Off |                    0 |
| N/A   33C    P0             58W /  300W |   19220MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import pandas as pd
import numpy as np
fol = "data/capture24/"
Y = np.load(fol + 'Y_Walmsley2020.npy')
T = np.load(fol + 'T.npy')
P = np.load(fol + 'P.npy')
X_feats = pd.read_pickle(fol + 'X_feats.pkl').values

In [4]:
print(np.unique(Y))

['light' 'moderate-vigorous' 'sedentary' 'sleep']


In [5]:
#Convert Y to int labels from string labels
mp = {y: i for i, y in enumerate(np.unique(Y))}
Y = np.array([mp[y] for y in Y])


In [6]:
reverse_mp = {i: y for y, i in mp.items()}
print(mp, reverse_mp)

{'light': 0, 'moderate-vigorous': 1, 'sedentary': 2, 'sleep': 3} {0: 'light', 1: 'moderate-vigorous', 2: 'sedentary', 3: 'sleep'}


In [ ]:
print(np.unique(Y))

[0 1 2 3]


In [8]:
print(X_feats.shape, Y.shape, T.shape, P.shape)

(934762, 32) (934762,) (934762,) (934762,)


In [9]:
# do test train val split by subject
import torch
def custom_train_test_split(P, test_size=0.1, random_state=42):
    #get all unique subjects
    P_unique = np.unique(P)
    rng = np.random.default_rng(random_state)
    tot = len(P_unique)
    test_size = int(tot * test_size)
    train_size = tot - test_size    
    train_subjects = rng.choice(P_unique, train_size, replace=False)
    test_subjects = np.setdiff1d(P_unique, train_subjects)
    train_ids = np.where(np.isin(P, train_subjects))[0]
    test_ids = np.where(np.isin(P, test_subjects))[0]
    return train_ids, test_ids

#do train test split by subject
train_ids, test_ids = custom_train_test_split(P, test_size=0.1, random_state=42)
#Also do a validation split by subject
train_ids, val_ids = custom_train_test_split(train_ids, test_size=0.1, random_state=42)

X_train, y_train, P_train = X_feats[train_ids], Y[train_ids], P[train_ids]
X_test, y_test, P_test = X_feats[test_ids], Y[test_ids], P[test_ids]
X_val, y_val, P_val = X_feats[val_ids], Y[val_ids], P[val_ids]

def normalize_data(X):
    X_mean = X.mean(axis=0)
    X_std = X.std(axis=0)
    return (X - X_mean) / X_std, X_mean, X_std

X_train, X_train_mean, X_train_std = normalize_data(X_train)
print(X_train_mean.shape, X_train_std.shape)
X_test = (X_test - X_train_mean) / X_train_std
X_val = (X_val - X_train_mean) / X_train_std

train_set = torch.utils.data.TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
test_set = torch.utils.data.TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())
val_set = torch.utils.data.TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())



(32,) (32,)


In [11]:
print(len(train_set), len(test_set), len(val_set))

762288 87776 84698


In [14]:
from run_experiment import run_expriment
import random
import string

#Comment this lines for relu


# activation = 'relu'
# enable_boundary_loss = False
# disable_batchnorm_grad_clip_exclusion = True
# lambda_penalty = -1
# gradient_clip_val = None



activation = 'poly'
enable_boundary_loss = True
disable_batchnorm_grad_clip_exclusion = False
lambda_penalty = 10
gradient_clip_val = 1.0


pol_degree = 4

pol_degree_map = {
    2:{"B": 13, "penalty_B": 13},
    4:{"B": 20, "penalty_B": 20 * 0.75},
    8:{"B": 35, "penalty_B": 35 * 0.5},
}

max_epoch = 10
dropout = 0.0
learning_rate = 0.001


project_name = "test"
data_workers = 4
model = "mlp"
dataset = {
    "train": train_set,
    "val": val_set,
    "test": test_set
}

run_id = "test_mlp"
custom_tag = "test"

ori_activaiton = "relu"
samp_size = 100

B = pol_degree_map[pol_degree]["B"]
penalty_B = pol_degree_map[pol_degree]["penalty_B"]


optimizer_params = {
        'type': 'adamw',
        'lr': learning_rate,
        'params': {
        }
}
scheduler_params = {'type': 'reduce_on_plateau',
                        'params': {
                                'mode': 'min',
                                'factor': 0.1,
                                'patience': 5,
                                'threshold': 0.1,
                                'verbose': True
                        },
                        'monitor': 'val_acc_epoch'
}
boundary_loss_params = {'type': 'exp', 'penalty_B':  penalty_B, 'acc_norm': 'sum'}
learnable_coeffs = True
input_size = X_train.shape[1]

boundary_loss_params = {'type': 'exp', 'penalty_B': B * 1.0, 'acc_norm': 'sum'}
actvation_params =  {
        "ori_activation": ori_activaiton,
        'B': B,
        'samp_size': samp_size,
        'pol_degree': pol_degree,
        'learnable_coeffs': learnable_coeffs,
        'initialization': "least_square",
        'boundary_loss_params': boundary_loss_params

    }
num_classes = np.unique(y_train).shape[0]
model_params = {
    "use_singleton_activation": False,
    "bn_before_act": False,
    "activation": activation,
    "dropout": dropout,
    "num_classes":num_classes,
    "actvation_params": actvation_params,
    "model":model,
    "input_size": input_size,
    'hidden_dims': [256, 128, 64, 32, 16]
}

training_params = {
    "enable_boundary_loss": enable_boundary_loss,
    "gradient_clip_val": gradient_clip_val,
    "max_epoch": max_epoch,
    "lambda_penalty": lambda_penalty,
    "disable_batchnorm_grad_clip_exclusion":disable_batchnorm_grad_clip_exclusion,
    'optimizer_params': optimizer_params,
    'scheduler_params': scheduler_params

}

dataset_params = {
    "data_workers": data_workers,
    "dataset": dataset,
    "batch_size": 800
}

project_params = {"run_id": run_id,
                  "project_name": project_name,
                  "custom_tag": custom_tag
                  }

run_expriment(project_params=project_params, dataset_params=dataset_params,
               model_params=model_params, training_params=training_params)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params | Mode 
------------------------------------------------------------
0 | model          | MLP              | 54.5 K | train
1 | base_criterion | CrossEntropyLoss | 0      | train
2 | criterion      | CustomPolyLoss   | 0      | train
------------------------------------------------------------
54.5 K    Trainable params
0         Non-trainable params
54.5 K    Total params
0.218     Total estimated model params size (MB)
30        Modules in train mode
0         Modules in eval mode


B 20 samp_size 100 initialization least_square pol_degree 4
B 20 samp_size 100 initialization least_square pol_degree 4
B 20 samp_size 100 initialization least_square pol_degree 4
B 20 samp_size 100 initialization least_square pol_degree 4
B 20 samp_size 100 initialization least_square pol_degree 4
B 20 samp_size 100 initialization least_square pol_degree 4


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Restoring states from the checkpoint path at /code/Polynomial-NN/notebooks/lightning_logs/version_40/checkpoints/best-acc-epoch=7-val_acc=0.00.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /code/Polynomial-NN/notebooks/lightning_logs/version_40/checkpoints/best-acc-epoch=7-val_acc=0.00.ckpt


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      Validate metric               DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       val_acc_epoch             0.7656969428062439
  val_boundary_loss_epoch        28.75305938720703
val_cross_entropy_loss_epoch     288.1406555175781
       val_loss_epoch            288.1406555175781
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Validation: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      Validate metric               DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       val_acc_epoch             0.7744144201278687
  val_boundary_loss_epoch      5.015075203118613e-06
val_cross_entropy_loss_epoch     0.5853334665298462
       val_loss_epoch            0.5853334665298462
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
